In [1]:
from functions import mle , gradient, init_graph, matching_func, regularized_vector, sigmoid, noise_generation

import random, csv, time, os, pickle, re, math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import optimize
from scipy.stats import bernoulli
from scipy.io import savemat
from __future__ import division
from datetime import datetime

%matplotlib inline

In [2]:
# file format
# file_name  real_age  apparent_age  worker_age  worker_gender

ReRead = False

if ReRead:
    path = './age/'
    csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]

    votes = dict()

    for files in csv_files:
        time0 = time.time()
        files = path+files
        df = pd.read_csv(files)
        for row in df.iterrows():
            if row[1]['file_name'] in votes.keys():
                votes[row[1]['file_name']]['votes'].append(row[1]['apparent_age'])
            else:
                votes[row[1]['file_name']] = dict()
                votes[row[1]['file_name']]['votes'] = [row[1]['apparent_age']]
                votes[row[1]['file_name']]['age'] = row[1]['real_age']
        print time.time()-time0
        
    for key in votes.keys():
        votes[key]['avg'] = np.mean(votes[key]['votes'])
    pickle.dump( votes, open( 'real_votes.p', "wb" ) )

In [3]:
# Real votes will be saved in the pickle file. In the future can directly load it
votes = pickle.load( open( 'real_votes.p', "rb" ) )

# Below is the code for pairs generate from individual votes and zero noise in generation. Use $\sigma = \sqrt2s$, where s is the standard deviation of the age data in the BLT model. 

In [19]:
items = votes.keys()
tot_num = len(items)
nlogn = int(tot_num*np.log(tot_num))
fractions = [0.5]
sigma = (4*np.sqrt(2))

for fraction in fractions:
    pairs = init_graph(tot_num, int(nlogn*fraction))
    
    for i in range(len(pairs)):
        if random.choice(votes[items[pairs[i][0]]]['votes']) < random.choice(votes[items[pairs[i][1]]]['votes']):
            pairs[i] = (pairs[i][1] ,pairs[i][0])
        
    start_time = time.time()
    w = np.ones(tot_num)
    res = optimize.minimize(mle, w, 
                            method='Newton-CG',
                            jac=gradient,
                            args=(pairs,sigma,),
                            tol = 10,
                            options={'disp': False})
    
    print time.time() - start_time

378.524878979


# Below is the code for pair generation with the average age for each person; use the same sigma for generation and fit in the BLT model

In [21]:
items = votes.keys()
tot_num = len(items)
nlogn = int(tot_num*np.log(tot_num))
fractions = [0.5]
sigma = (4*np.sqrt(2))

for fraction in fractions:
    pairs = init_graph(tot_num, int(nlogn*fraction))
    
    for i in range(len(pairs)):
        if votes[items[pairs[i][0]]]['avg'] < votes[items[pairs[i][1]]]['avg']:
            pairs[i] = (pairs[i][1] ,pairs[i][0])
        
    start_time = time.time()
    w = np.ones(tot_num)
    res1 = optimize.minimize(mle, w, 
                            method='Newton-CG',
                            jac=gradient,
                            args=(pairs,sigma,),
                            tol = 10,
                            options={'disp': False})
    
    print time.time() - start_time

407.040500879


In [16]:
def R2(yhat, y):
    ybar = np.sum(y)/len(y) 
    ssreg = np.sum((yhat-ybar)**2)
    sstot = np.sum((y - ybar)**2)
    return ssreg/sstot

w_star = []
w_hat = []
w_apparent = []

for i in range(len(items)):
    w_star.append(votes[items[i]]['age'])
    w_apparent.append(np.mean(votes[items[i]]['votes']))
w_hat_raw = res['x']
w_hat = regularized_vector(w_star,w_hat_raw)

R2(w_hat, w_apparent)

0.78712430916044962

In [20]:
def R2(yhat, y):
    ybar = np.sum(y)/len(y) 
    ssreg = np.sum((yhat-ybar)**2)
    sstot = np.sum((y - ybar)**2)
    return ssreg/sstot

w_star = []
w_hat = []
w_apparent = []

for i in range(len(items)):
    w_star.append(votes[items[i]]['age'])
    w_apparent.append(np.mean(votes[items[i]]['votes']))
w_hat_raw = res['x']
w_hat = regularized_vector(w_star,w_hat_raw)

R2(w_hat, w_apparent)

0.79074917106450648

In [22]:
def R2(yhat, y):
    ybar = np.sum(y)/len(y) 
    ssreg = np.sum((yhat-ybar)**2)
    sstot = np.sum((y - ybar)**2)
    return ssreg/sstot

w_star = []
w_hat = []
w_apparent = []

for i in range(len(items)):
    w_star.append(votes[items[i]]['age'])
    w_apparent.append(np.mean(votes[items[i]]['votes']))
w_hat_raw = res1['x']
w_hat = regularized_vector(w_star,w_hat_raw)

R2(w_hat, w_apparent)

0.79504486961474141